In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [3]:
data=pd.read_csv('bankmodels.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,Portland,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Salem,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,Portland,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,Portland,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Salem,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
### Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Portland,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Salem,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,Portland,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,Portland,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Salem,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,Portland,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,Portland,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,Portland,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Tigard,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## Encode the gender column to 0 and 1
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
#convert the geo location features
## Onehot encode 'Geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_Portland,Geography_Salem,Geography_Tigard
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,0.0,1.0


In [7]:
## Combine one hot encoder columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Portland,Geography_Salem,Geography_Tigard
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0,0.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0,0.0


In [9]:
#split the salary data features, put estimatedsalary into y, everything else into X
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [10]:
## Split the data in training and tetsing sets before we use scaler.pki
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [22]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


<h2> ANN Regression problem statement

In [13]:
import tensorflow as tf
print(tf.__version__)

from tf_keras.models import Sequential
from tf_keras.layers import Dense
from tf_keras.optimizers.legacy import Adam

2.18.0


In [14]:
# Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1
    Dense(32,activation='relu'), ## HL2
    Dense(1)  ## output layer no activation function applied, default used - linear activation
  ]
)

#compile the model
opt = Adam()

model.compile(
  optimizer=opt,
  loss="mean_absolute_error",
  metrics=['mae']
)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
#setup the tensorboard and logs
import datetime
from tf_keras.callbacks import EarlyStopping, TensorBoard

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [16]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(
  monitor='val_loss',
  patience=5,
  restore_best_weights=True
)


In [17]:
#train the model
history=model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)


Epoch 1/100
250/250 [==============================] - 0s 634us/step - loss: 100376.4609 - mae: 100376.4609 - val_loss: 98509.1328 - val_mae: 98509.1328
Epoch 2/100
250/250 [==============================] - 0s 447us/step - loss: 99584.5625 - mae: 99584.5625 - val_loss: 96900.7734 - val_mae: 96900.7734
Epoch 3/100
250/250 [==============================] - 0s 445us/step - loss: 96785.1016 - mae: 96785.1016 - val_loss: 92791.1797 - val_mae: 92791.1797
Epoch 4/100
250/250 [==============================] - 0s 432us/step - loss: 91272.1953 - mae: 91272.1953 - val_loss: 85973.8906 - val_mae: 85973.8906
Epoch 5/100
250/250 [==============================] - 0s 442us/step - loss: 83321.6953 - mae: 83321.6953 - val_loss: 77430.8125 - val_mae: 77430.8125
Epoch 6/100
250/250 [==============================] - 0s 436us/step - loss: 74261.6172 - mae: 74261.6172 - val_loss: 68547.0234 - val_mae: 68547.0234
Epoch 7/100
250/250 [==============================] - 0s 433us/step - loss: 65598.6641 - ma

In [18]:
## Load Tensorboard Extension
%load_ext tensorboard

In [19]:
%tensorboard --logdir regressionlogs/fit

In [20]:
#evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae:.2f}')

63/63 [==============================] - 0s 276us/step - loss: 50313.5508 - mae: 50313.5508
Test MAE: 50313.55


In [21]:
#save the model
model.save('regression_model.h5')

/Users/chrism/Projects/AIProjects/.venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
